In [2]:
from pyscipopt import Model
import numpy as np

# Practica 1

## Ejercicio 1

Features
- Cantidad de porciones: con i perteneciente a [1, 2, 3]
- Cantidad de vitaminas: con j perteneciente a [A, C, D]

Variables
- A_i - Cantidad de porciones de 30g del alimento i 

Datos
- V_ij - Cantidad de vitamina j en una porcion de 30 gramos del alimento i
- C_i - Costo de 30 gramos del alimento i
- RDM_j - racion diaria minima de vitamina j

FO: 
- MIN(COSTO = SUM(A_i * C_I))
s.a: 
- SUM_i(A_i) >= 225/30  
- SUM_i(A_I * V_ij) >= RDM_j for all j

In [13]:
# Vitamin content per 30g [rows: A, C, D; columns: Alimentos 1, 2, 3]
vitamin_content = np.array([
    [0.03, 0.02, 0.04],   # Vitamina A
    [0.01, 0.015, 0.005], # Vitamina C
    [0.04, 0.03, 0.02]    # Vitamina D
])

# Ración Diaria Mínima (RDM) for vitamins A, C, D
rdm = np.array([0.3, 0.12, 0.21])

# Cost per 30g portion for each food
cost = np.array([0.15, 0.10, 0.12])

# Portion size per unit and total minimum required in grams
portion_size = 30
min_total_grams = 225
min_total_portions = min_total_grams / portion_size  # 7.5 portions minimum




model = Model('Ej1')

food_portion = np.empty(len(cost), dtype='object')
for i in range(len(cost)):
  food_portion[i] = model.addVar(f'x_{i}', vtype='I')

model.setObjective(sum(food_portion[i] * cost[i] for i in range(len(cost))), sense='minimize')

model.addCons(sum(food_portion[i] for i in range(len(cost))) >= min_total_portions)

for j in range(len(rdm)):
    model.addCons(sum(food_portion[i] * vitamin_content[j, i] for i in range(len(cost))) >= rdm[j])

model.redirectOutput()
model.optimize()
sol = model.getBestSol()
model.writeSol(sol, './InvestigacionOperativa/practica/practica1/Ej1.sol')

feasible solution found by trivial heuristic after 0.0 seconds, objective value 3.700000e+04
presolving:
(round 1, fast)       0 del vars, 0 del conss, 0 add conss, 3 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       0 del vars, 0 del conss, 0 add conss, 6 chg bounds, 1 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
   (0.0s) symmetry computation started: requiring (bin +, int +, cont +), (fixed: bin -, int -, cont -)
   (0.0s) no symmetry present (symcode time: 0.00)
presolving (3 rounds: 3 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 6 tightened bounds, 0 added holes, 1 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 3 variables (0 bin, 3 int, 0 impl, 0 cont) and 4 constraints
      4 constraints of type <linear>
transformed objective value is always integral (scale: 0.01)
Presolving Time: 0.00

 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |va

In [23]:
print(f"costo final: {sum(sol[food_portion[i]] * cost[i] for i in range(len(food_portion)))}")
print(f"req: {225/30}; cantidad porciones: {sum(sol[food_portion[i]] for i in range(len(food_portion)))}")

for j in range(len(rdm)):
    print(f"req: {rdm[j]}; vitamina {j}: {sum(sol[food_portion[i]] * vitamin_content[j,i] for i in range(len(food_portion)))}")

costo final: 1.1800000000000002
req: 7.5; cantidad porciones: 11.0
req: 0.3; vitamina 0: 0.30000000000000004
req: 0.12; vitamina 1: 0.125
req: 0.21; vitamina 2: 0.29


## Ejercicio 2

Features
- Cantidad de porciones: con i perteneciente a [1, 2, 3]
- Cantidad de vitaminas: con j perteneciente a [A, C, D]

Variables
- A_i - Cantidad de porciones de 30g del alimento i 

Datos
- V_ij - Cantidad de vitamina j en una porcion de 30 gramos del alimento i
- C_i - Costo de 30 gramos del alimento i
- RDM_j - racion diaria minima de vitamina j

FO: 
- MIN(COSTO = SUM(A_i * C_I))
s.a: 
- SUM_i(A_i) >= 225/30  
- SUM_i(A_I * V_ij) >= RDM_j for all j